<a href="https://colab.research.google.com/github/spencer18001/Clustering-And-Dimensionality-Reduction---Deep-Dive/blob/main/07/0710.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!wget -O 0710_soybean_data_use.csv https://github.com/spencer18001/Clustering-And-Dimensionality-Reduction---Deep-Dive/blob/main/data/0710_soybean_data_use.csv?raw=1

## Categorical features - encoding and distances

In [6]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import pairwise_distances

from scipy.spatial.distance import hamming, euclidean, pdist, squareform

## Soyabeen small dataset

A dataset of soybean plant observations, including information on plants infested by one of four diseases.

- Dataset contains 47 instances.
- Each instance represents a single plant.
- Characterized by 35 attributes.
- Attributes are categorical.
- Attributes mostly capture various symptoms like leaf spots, root rot, mold growth, seed damage, etc.

Citation:

- Michalski,R.. (1987). Soybean (Small). UCI Machine Learning Repository. https://doi.org/10.24432/C5DS3P.

First 2 features:

1. **Date**: The time when the soybean sample was collected. May be represented as a date or as the day of the year.
  
2. **Hail**: Indicates whether the plants have been affected by hail, generally a binary "yes" or "no."

3. **Germination**: Describes the rate of germination.

In [7]:
soybean_path = '0710_soybean_data_use.csv'
soy_df = pd.read_csv(soybean_path)
soy_df = soy_df.loc[:10, ['date', 'hail', 'germination']]
soy_df

,date,hail,germination
0,august,no,lt-80%
1,september,yes,lt-80%
2,july,yes,80-89%
3,october,yes,90-100%
4,august,yes,lt-80%
5,september,yes,90-100%
6,july,yes,80-89%
7,july,yes,lt-80%
8,october,yes,80-89%
9,october,yes,lt-80%


In [8]:
soy_df['date'].unique()

array(['august', 'september', 'july', 'october'], dtype=object)

In [9]:
soy_df['hail'].unique()

array(['no', ' yes'], dtype=object)

In [10]:
soy_df['germination'].unique()

array(['lt-80%', '80-89%', ' 90-100%'], dtype=object)

### Hamming distance

In [11]:
soy_df.loc[:1, :]

,date,hail,germination
0,august,no,lt-80%
1,september,yes,lt-80%


In [12]:
hamming(soy_df.loc[0].to_numpy(), soy_df.loc[1].to_numpy()) # normalized Hamming distance

0.6666666666666666

In [13]:
dst = pdist(soy_df.to_numpy(), metric='hamming') # without encoding the data first
dst_matrix = squareform(dst)
pd.DataFrame(dst_matrix)

ValueError: Unsupported dtype object

In [14]:
soy_df

,date,hail,germination
0,august,no,lt-80%
1,september,yes,lt-80%
2,july,yes,80-89%
3,october,yes,90-100%
4,august,yes,lt-80%
5,september,yes,90-100%
6,july,yes,80-89%
7,july,yes,lt-80%
8,october,yes,80-89%
9,october,yes,lt-80%


In [15]:
or_encoder = OrdinalEncoder()
soy_df_enc = or_encoder.fit_transform(soy_df)
soy_df_enc # datatype: float

array([[0., 1., 2.],
       [3., 0., 2.],
       [1., 0., 1.],
       [2., 0., 0.],
       [0., 0., 2.],
       [3., 0., 0.],
       [1., 0., 1.],
       [1., 0., 2.],
       [2., 0., 1.],
       [2., 0., 2.],
       [2., 1., 0.]])

In [16]:
dst = pdist(soy_df_enc, metric='hamming')
dst

array([0.66666667, 1.        , 1.        , 0.33333333, 1.        ,
       1.        , 0.66666667, 1.        , 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.33333333, 0.33333333, 0.66666667,
       0.33333333, 0.66666667, 0.33333333, 1.        , 0.66666667,
       0.66666667, 0.66666667, 0.        , 0.33333333, 0.33333333,
       0.66666667, 1.        , 0.66666667, 0.33333333, 0.66666667,
       0.66666667, 0.33333333, 0.33333333, 0.33333333, 0.66666667,
       0.66666667, 0.33333333, 0.66666667, 0.33333333, 1.        ,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.33333333, 0.33333333, 0.66666667, 1.        , 0.66666667,
       0.33333333, 1.        , 0.33333333, 0.66666667, 0.66666667])

In [17]:
dst_matrix = squareform(dst)
pd.DataFrame(dst_matrix) # pairwise distance matrix

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.666667,1.000000,1.000000,0.333333,1.000000,1.000000,0.666667,1.000000,0.666667,0.666667
1,0.666667,0.000000,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.666667,0.333333,1.000000
2,1.000000,0.666667,0.000000,0.666667,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,1.000000
3,1.000000,0.666667,0.666667,0.000000,0.666667,0.333333,0.666667,0.666667,0.333333,0.333333,0.333333
4,0.333333,0.333333,0.666667,0.666667,0.000000,0.666667,0.666667,0.333333,0.666667,0.333333,1.000000
5,1.000000,0.333333,0.666667,0.333333,0.666667,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667
6,1.000000,0.666667,0.000000,0.666667,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,1.000000
7,0.666667,0.333333,0.333333,0.666667,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,1.000000
8,1.000000,0.666667,0.333333,0.333333,0.666667,0.666667,0.333333,0.666667,0.000000,0.333333,0.666667
9,0.666667,0.333333,0.666667,0.333333,0.333333,0.666667,0.666667,0.333333,0.333333,0.000000,0.666667


In [18]:
dst_matrix1 = pairwise_distances(soy_df_enc, metric='hamming')
pd.DataFrame(dst_matrix) # by sklearn

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.666667,1.000000,1.000000,0.333333,1.000000,1.000000,0.666667,1.000000,0.666667,0.666667
1,0.666667,0.000000,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.666667,0.333333,1.000000
2,1.000000,0.666667,0.000000,0.666667,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,1.000000
3,1.000000,0.666667,0.666667,0.000000,0.666667,0.333333,0.666667,0.666667,0.333333,0.333333,0.333333
4,0.333333,0.333333,0.666667,0.666667,0.000000,0.666667,0.666667,0.333333,0.666667,0.333333,1.000000
5,1.000000,0.333333,0.666667,0.333333,0.666667,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667
6,1.000000,0.666667,0.000000,0.666667,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,1.000000
7,0.666667,0.333333,0.333333,0.666667,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,1.000000
8,1.000000,0.666667,0.333333,0.333333,0.666667,0.666667,0.333333,0.666667,0.000000,0.333333,0.666667
9,0.666667,0.333333,0.666667,0.333333,0.333333,0.666667,0.666667,0.333333,0.333333,0.000000,0.666667


In [19]:
np.array_equal(dst_matrix, dst_matrix1)

True

### Euclidean distance

In [24]:
soy_df

,date,hail,germination
0,august,no,lt-80%
1,september,yes,lt-80%
2,july,yes,80-89%
3,october,yes,90-100%
4,august,yes,lt-80%
5,september,yes,90-100%
6,july,yes,80-89%
7,july,yes,lt-80%
8,october,yes,80-89%
9,october,yes,lt-80%


In [20]:
oh_encoder = OneHotEncoder(sparse_output=False)
soy_df_oh_enc = oh_encoder.fit_transform(soy_df)

In [21]:
soy_df.nunique()

,0
date,4
hail,2
germination,3


In [22]:
soy_df_oh_enc

array([[1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 1., 0., 0.]])

In [25]:
soy_df_oh_enc.shape

(11, 9)

In [26]:
euclidean(soy_df_oh_enc[0,:], soy_df_oh_enc[1,:])

2.0

In [27]:
dst = pdist(soy_df_oh_enc, metric='euclidean')
dst_matrix = squareform(dst)
pd.DataFrame(dst_matrix)

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,2.000000,2.449490,2.449490,1.414214,2.449490,2.449490,2.000000,2.449490,2.000000,2.000000
1,2.000000,0.000000,2.000000,2.000000,1.414214,1.414214,2.000000,1.414214,2.000000,1.414214,2.449490
2,2.449490,2.000000,0.000000,2.000000,2.000000,2.000000,0.000000,1.414214,1.414214,2.000000,2.449490
3,2.449490,2.000000,2.000000,0.000000,2.000000,1.414214,2.000000,2.000000,1.414214,1.414214,1.414214
4,1.414214,1.414214,2.000000,2.000000,0.000000,2.000000,2.000000,1.414214,2.000000,1.414214,2.449490
5,2.449490,1.414214,2.000000,1.414214,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
6,2.449490,2.000000,0.000000,2.000000,2.000000,2.000000,0.000000,1.414214,1.414214,2.000000,2.449490
7,2.000000,1.414214,1.414214,2.000000,1.414214,2.000000,1.414214,0.000000,2.000000,1.414214,2.449490
8,2.449490,2.000000,1.414214,1.414214,2.000000,2.000000,1.414214,2.000000,0.000000,1.414214,2.000000
9,2.000000,1.414214,2.000000,1.414214,1.414214,2.000000,2.000000,1.414214,1.414214,0.000000,2.000000


In [28]:
dst_matrix1 = pairwise_distances(soy_df_oh_enc, metric='euclidean')
pd.DataFrame(dst_matrix)

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,2.000000,2.449490,2.449490,1.414214,2.449490,2.449490,2.000000,2.449490,2.000000,2.000000
1,2.000000,0.000000,2.000000,2.000000,1.414214,1.414214,2.000000,1.414214,2.000000,1.414214,2.449490
2,2.449490,2.000000,0.000000,2.000000,2.000000,2.000000,0.000000,1.414214,1.414214,2.000000,2.449490
3,2.449490,2.000000,2.000000,0.000000,2.000000,1.414214,2.000000,2.000000,1.414214,1.414214,1.414214
4,1.414214,1.414214,2.000000,2.000000,0.000000,2.000000,2.000000,1.414214,2.000000,1.414214,2.449490
5,2.449490,1.414214,2.000000,1.414214,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
6,2.449490,2.000000,0.000000,2.000000,2.000000,2.000000,0.000000,1.414214,1.414214,2.000000,2.449490
7,2.000000,1.414214,1.414214,2.000000,1.414214,2.000000,1.414214,0.000000,2.000000,1.414214,2.449490
8,2.449490,2.000000,1.414214,1.414214,2.000000,2.000000,1.414214,2.000000,0.000000,1.414214,2.000000
9,2.000000,1.414214,2.000000,1.414214,1.414214,2.000000,2.000000,1.414214,1.414214,0.000000,2.000000
